In [1]:
!python -m pip install --upgrade pip
!pip install music21


In [2]:
!pip install glob2

In [3]:
from music21 import converter, instrument, note, chord

notes = []

In [5]:
import glob 

for file in glob.glob("/workspaces/codespaces-jupyter/data/midi_songs/*.mid"): #Used to read files from a folder with .mid extension
    midi = converter.parse(file)
    notes_to_parse = None

    parts = instrument.partitionByInstrument(midi)
    if parts:
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
    


In [6]:
for element in notes_to_parse:
    if isinstance(element,note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

In [7]:
sequence_length = 100

pitchnames = sorted(set(item for item in notes))

In [8]:
print(pitchnames)

['0.2', '0.2.5', '0.3.7', '0.4', '0.4.7', '0.5', '10.0.5', '10.2', '10.2.5', '3.6', '3.7', '3.7.10', '3.9', '5.7.10', '5.8.0', '5.9', '5.9.0', '6.10.1', '6.9.0', '7.10', '7.10.2', '7.11.0', '7.11.2', '9.0', 'A3', 'A4', 'A5', 'B-2', 'B-3', 'B-4', 'B-5', 'B3', 'B4', 'C#3', 'C3', 'C4', 'C5', 'C6', 'D3', 'D4', 'D5', 'D6', 'E-3', 'E-4', 'E-5', 'E-6', 'F#3', 'F3', 'F5', 'F6', 'G#2', 'G3', 'G4', 'G5', 'G6']


In [9]:
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

In [10]:
print(note_to_int)

{'0.2': 0, '0.2.5': 1, '0.3.7': 2, '0.4': 3, '0.4.7': 4, '0.5': 5, '10.0.5': 6, '10.2': 7, '10.2.5': 8, '3.6': 9, '3.7': 10, '3.7.10': 11, '3.9': 12, '5.7.10': 13, '5.8.0': 14, '5.9': 15, '5.9.0': 16, '6.10.1': 17, '6.9.0': 18, '7.10': 19, '7.10.2': 20, '7.11.0': 21, '7.11.2': 22, '9.0': 23, 'A3': 24, 'A4': 25, 'A5': 26, 'B-2': 27, 'B-3': 28, 'B-4': 29, 'B-5': 30, 'B3': 31, 'B4': 32, 'C#3': 33, 'C3': 34, 'C4': 35, 'C5': 36, 'C6': 37, 'D3': 38, 'D4': 39, 'D5': 40, 'D6': 41, 'E-3': 42, 'E-4': 43, 'E-5': 44, 'E-6': 45, 'F#3': 46, 'F3': 47, 'F5': 48, 'F6': 49, 'G#2': 50, 'G3': 51, 'G4': 52, 'G5': 53, 'G6': 54}


In [11]:
network_input = []
network_output = []

In [12]:
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i : i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

The sequence in contains inputs like 1 to n, 2 to n and so on as it is  a recurrent neural network

In [13]:
!pip install keras
!pip install tensorflow


In [14]:
import numpy as np

n_vocab = len(set(notes))
n_patterns  = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))

network_input = network_input/float(n_vocab)


In [15]:
import tensorflow as tf

network_output = tf.keras.utils.to_categorical(network_output)


2023-12-21 16:27:36.973384: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 16:27:39.172911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-21 16:27:39.173011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-21 16:27:39.495587: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-21 16:27:40.211198: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-21 16:27:40.212467: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation

In [17]:
model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1],network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [38]:
import tensorflow as tf
 
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
callbacks_list = [checkpoint]

model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200


5/5 [==============================] - 14s 2s/step - loss: 4.1456
Epoch 2/200


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5/5 [==============================] - 11s 2s/step - loss: 3.6276
Epoch 3/200
5/5 [==============================] - 11s 2s/step - loss: 3.5221
Epoch 4/200
5/5 [==============================] - 11s 2s/step - loss: 3.5085
Epoch 5/200
5/5 [==============================] - 11s 2s/step - loss: 3.4993
Epoch 6/200
5/5 [==============================] - 11s 2s/step - loss: 3.4789
Epoch 7/200
5/5 [==============================] - 11s 2s/step - loss: 3.5129
Epoch 8/200
5/5 [==============================] - 12s 2s/step - loss: 3.4486
Epoch 9/200
5/5 [==============================] - 11s 2s/step - loss: 3.7928
Epoch 10/200
5/5 [==============================] - 11s 2s/step - loss: 3.4075
Epoch 11/200
5/5 [==============================] - 11s 2s/step - loss: 3.4214
Epoch 12/200
5/5 [==============================] - 11s 2s/step - loss: 3.3448
Epoch 13/200
5/5 [==============================] - 11s 2s/step - loss: 3.2127
Epoch 14/200
5/5 [==============================] - 11s 2s/step - loss: 

In [39]:
model.save_weights('../weights.hdf5')


In [ ]:
model.load_weights('../weights.hdf5')

In [18]:
start = np.random.randint(0, len(network_input)-1)

In [29]:
pattern = network_input[start]
prediction_output = []

In [30]:
int_to_note = dict((number, note) for number, note in enumerate(pitchnames) )

In [32]:
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1,len(pattern),1))
    prediction_input = prediction_input/float(n_vocab)
    prediction = model.predict(prediction_input)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)

    pattern = np.append(pattern, index)
    pattern = pattern[1:len(pattern)]

1/1 [==============================] - 0s 50ms/step


1/1 [==============================] - 0s 49ms/step


In [34]:
offset = 0
output_notes = []

for pattern in prediction_output:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    offset+=0.5

In [36]:
from music21 import stream
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'